# The website wizards 

## Our website: 
### https://ri-ru.github.io/ADA-website/

In the following, we look at our design process for the website, and focus on some key moments that shaped our journey :-)


# Chapter 1: Inspiration for our design

### Initial inspiration and sketches 

The first part of the job was very important, as it would set the tone for the rest of our datastory. As a large team effort, we discussed a great deal about what datastory we wanted. We looked at some examples together to reference our website design and flow, we created sketches, drew inspiration from others, and made mockups. It did not look good at first, and it was hard to communicate what the website should become.But, we did have a general idea of the structure.


<img src="assets/brainstorming/CanvaDraft.png" 
        alt="Picture" 
        width="300" 
        height="600" 
        style="display: block; margin: 0 auto" />

Here's some beautiful datastories and graphics we used as inspiration:


https://pudding.cool/2021/04/music-bubble/    |  [www.denizcemonduygu.com/](https://www.denizcemonduygu.com/) | https://www.karim.news/
:-------------------------:|:-------------------------:|:-------------------------:
![](assets/brainstorming/Ex1.png)  |  ![](assets/brainstorming/Ex2.png) | ![](assets/brainstorming/Ex3.png)




# Chapter 2: The nitty-gritty of the website - hosting

<div style="overflow: auto;">
    <img src="static\assets\pixel_art\Jan.gif" 
         alt="Pixel Art" 
         style="float: left; margin-left: 0px; margin-bottom: 10px; margin-top: 12px; margin-right: 10px;  width: 80px; height: 80px; image-rendering: pixelated;" /> 
         
**Question: how to create and host a website if you have never done it before?**

We initially explored Jekyll for hosting, but it wasn't ideal for our continuous narrative flow—it's better suited for blog-style posts. We wanted something more dynamic and interactive to tell our story.

Eventually, building on a template used by our inspirations, we settled on SvelteKit (Svelte 5). Despite the initial learning curve, it seems like certainly the right decision now! Without boring you with how Svelte is better than other frameworks (hint: it is)... Unlike frameworks like React that ship their runtime to the browser, Svelte compiles components to optimized vanilla JavaScript at build time. So, smaller bundles and faster load times, all with incredible flexibility and reactivity! 

For hosting, we use GitHub Pages, so we don't need to host our own server nor have any database. The build process outputs pure HTML/CSS/JS files to the build directory, which GitHub serves directly. Very convenient!

Below is our SvelteKit config for GitHub Pages hosting.

```svelte
const config = {
	compilerOptions: {
		runes: true
	},
	preprocess,
	kit: {
		adapter: adapterStatic({
			pages: 'build',
			assets: 'build',
			fallback: 'index.html',
			precompress: false,
			strict: false
		}),
		paths: {
			base: dev ? '' : '/ADA-website'
		}
	}
};

export default config;

# Chapter 3: Snippets of Implementations

###  Our narrative - the explanation 

<div style="overflow: auto;">
    <img src="static\assets\pixel_art\Jan.gif" 
         alt="Pixel Art" 
         style="float: left; margin-left: 0px; margin-bottom: 10px; margin-top: 12px; margin-right: 10px;  width: 80px; height: 80px; image-rendering: pixelated;" />  

Our datastory narrative is about a Youtuber named Jimmy, who recently started his channel and wants to get paid for it. He finds a "get rich quick scheme" course online, that looks quite suspicious, but he borrows his mom's credit card anyway and buys the course. Instead, surprise: This course is actually a collection of skilled analysts, and provide high quality data analysis to help Jimmy understand what kind of Youtuber he needs to become to also make money from his channel! 

To tell this story and allow the reader to get lost in the narrative (hopefully ;)), we personify Jimmy through a low quality video, and through a serious of texting messages between him and the analysts. The story is meant to be light and easy to read, with stretches of analyses followed by some silly jokes. We show that there is real interaction between the analysts and Jimmy as they show and describe plots to which Jimmy asks questions if he's curious. 

Jimmy's video: 

<img src="assets/implementation/JimmyVid.png" 
        alt="Picture" 
        width="300" 
        height="200" 
        style="display: block; margin: 0 auto" />

### Data Storage

To make every member's analysis addition easier, we used ArchieML with Google Docs. This way, any member could add their own text messages without needing to touch the code, no pushing tiny changes, all was synced at once.

How it works:
- Content lives in Google Docs to be added/edited
- Synced to JSON & CSV
- Parsed into chat bubbles with corresponding avatars

Team members write dialogue like this:

<img src="assets/implementation/Sheets.png" 
        alt="Picture" 
        width="800" 
        height="300" 
        style="display: block; margin: 0 auto" />

Our generated json is like so:
   ```json
   {
     "intro_chat_1": "[eddie] Oh... a customer?",
     "intro_chat_2": "[jimmy] I just spent 999$ on my mom's credit card...",
     "intro_chat_7": "[vivian] Wait! What we CAN do is show you..."
   }
   ```




### The Dialogue System

In our main file Index.svelte, the  `parseChatMessage()` function uses regex to extract the speakers name from the Json above#, the message and the placement on which side to put the speaker (Jimmy on the right)


   ```javascript
   function parseChatMessage(str) {
     const match = str.match(/^\[(\w+)\]\s*(.*)$/);
     if (match) {
       const speakerKey = match[1].toLowerCase();
       const text = match[2];
       const side = speakerKey === 'jimmy' ? 'right' : 'left';
       return { speaker: match[1], text, side };
     }
     return { speaker: "Unknown", text: str, side: "left" };
   }
   ```


We then have to display the pixel art, support the HTML formatting, and making cute speech bubbles. 

To configure the speakers, we define each team member with the correspoinding pixel avatar: 

```javascript
const speakers = {
    jimmy: { avatar: "assets/pixel_art/jan.gif", name: "Jimmy" },
    eddie: { avatar: "assets/pixel_art/ender.gif", name: "Eddie" },
    kaleb: { avatar: "assets/pixel_art/kalan.gif", name: "Kaleb" },
    daniel: { avatar: "assets/pixel_art/danael.gif", name: "Daniel" },
    vivian: { avatar: "assets/pixel_art/veronika.gif", name: "Vivian" },
    siba: { avatar: "assets/pixel_art/siba.gif", name: "Siba" }
};

function getSpeaker(id) {
    return speakers[id.toLowerCase()] || 
           { avatar: "assets/pixel_art/default.gif", name: id };
}
```

Using Svelte 5's `{#snippet}` syntax, we created a reusable dialogue component that handles message rendering:


```svelte
{#snippet dialogue(messages)}
    <div class="dialogue">
        {#each messages as msg, i}
            <div class="msg {msg.side}" style="--delay: {i * 0.1}s">
                {#if msg.side === "left"}
                    <!-- Experts on LEFT -->
                    <div class="avatar-col">
                        <img class="avatar-img" 
                             src={getSpeaker(msg.speaker).avatar} 
                             alt={msg.speaker} />
                        <span class="speaker-name">
                            {getSpeaker(msg.speaker).name}
                        </span>
                    </div>
                    <div class="bubble bubble-left">
                        <p>{@html msg.text}</p>
                    </div>
                {:else}
                    <!-- Jimmy on RIGHT -->
                    <div class="bubble bubble-right">
                        <p>{@html msg.text}</p>
                    </div>
                    <div class="avatar-col">
                        <img class="avatar-img" 
                             src={getSpeaker(msg.speaker).avatar} 
                             alt={msg.speaker} />
                        <span class="speaker-name">
                            {getSpeaker(msg.speaker).name}
                        </span>
                    </div>
                {/if}
            </div>
        {/each}
    </div>
{/snippet}
```


With these tools, we can render all sorts of dialogues and texts! 

### 3. Crisp Pixel Art Rendering

To keep pixel art sharp (not blurry), we use specific CSS properties:

```css
.avatar-img {
    width: 76px;
    height: 80px;
    image-rendering: pixelated;
    image-rendering: crisp-edges;
    -webkit-image-rendering: pixelated;
    transform: translateZ(0);
}
```

The `image-rendering: pixelated` prevents the browser from smoothing our pixel art, and Jimmy's avatar gets `transform: scaleX(-1)` to mirror it for visual variety.

In [ ]:

const speakers = {
    jimmy: { avatar: "assets/pixel_art/jan.gif", name: "Jimmy" }
};

function getSpeaker(id) {
    return speakers[id.toLowerCase()] || { avatar: "assets/pixel_art/default.gif", name: id };
}

function parseChatMessage(str) {
    const match = str.match(/^\[(\w+)\]\s*(.*)$/);
    if (match) {
        const speakerKey = match[1].toLowerCase();
        const text = match[2];
        const side = speakerKey === 'jimmy' ? 'right' : 'left';
        return { speaker: match[1], text, side };
    }
    return { speaker: "Unknown", text: str, side: "left" };
}


##<-- dialogue rendering component -->

{#snippet dialogue(messages)}
    <div class="dialogue">
        {#each messages as msg, i}
            <div class="msg {msg.side}" style="--delay: {i * 0.1}s">
                {#if msg.side === "left"}
                    <!-- Expert on LEFT -->
                    <div class="avatar-col">
                        <img class="avatar-img" src={getSpeaker(msg.speaker).avatar} alt={msg.speaker} />
                        <span class="speaker-name">{getSpeaker(msg.speaker).name}</span>
                    </div>
                    <div class="bubble bubble-left">
                        <p>{@html msg.text}</p>
                    </div>
                {:else}
                    <!-- Jimmy on RIGHT -->
                    <div class="bubble bubble-right">
                        <p>{@html msg.text}</p>
                    </div>
                    <div class="avatar-col">
                        <img class="avatar-img" src={getSpeaker(msg.speaker).avatar} alt={msg.speaker} />
                        <span class="speaker-name">{getSpeaker(msg.speaker).name}</span>
                    </div>
                {/if}
            </div>
        {/each}
    </div>
{/snippet}

#using it: 

{@render dialogue(introChat)}

SyntaxError: invalid syntax (2917671061.py, line 1)

## The design & pixel art 

To draw the avatars, we used this website: https://www.pixilart.com/draw to draw them by hand.

### Overall design

The main problem was to make the website crisp and as consistent as possible. 

We established a unified color system that was necessary as each plot differs, using a Style Dictionary to maintain color consistency across CSS and JavaScript. We define design tokens once, and then generate them for both environments.



Source of Truth: design_system.css:




In [ ]:
#making it crisp 

.avatar-img {
    width: 76px;
    height: 80px;
    min-width: 76px;
    min-height: 80px;
    flex-shrink: 0;
    image-rendering: pixelated;
    image-rendering: crisp-edges;
    -webkit-image-rendering: pixelated;
    transform: translateZ(0);
}

## Category Comparisons

A quick example of our colour consistency. 

Users pick "Category A" and "Category B", and the charts need to update synchronously, the colors need to stay consistent, and the scales need to be comparable.

The Trick is oto use Svelte5 runes ($state, $effect), whcih makes this reactive. When you change something, the charts instantly update.

MADE THE BELOW PRETTIER - ITS NOT QUITE THERE YET. 

In [ ]:
let selectedCategory = $state(null);
let vizIframe = $state(null);

// Compare categories
let compareCatA = $state(null);
let compareCatB = $state(null);
let compareIframe = $state(null);

function selectCategory(cat) {
    selectedCategory = cat;
    // Send message to iframe to update visualization
    if (vizIframe && vizIframe.contentWindow) {
        vizIframe.contentWindow.postMessage({
            type: "selectCategory",
            category: cat ? cat.name : null
        }, "*");
    }
}

function selectCompareA(cat) {
    compareCatA = cat;
    sendCompareMessage();
}

function selectCompareB(cat) {
    compareCatB = cat;
    sendCompareMessage();
}

SyntaxError: invalid syntax (301563404.py, line 1)

## How charts are rendered 

We imported svelte components directly, and then use them. There are two chart types, echarts & chart js. Here's how:

In [ ]:
import KalanChart0 from "./KalanChart0.svelte"
import KalanChart1 from "./KalanChart1.svelte"
import Compare_1 from "./Compare_1.svelte"
import VisualizationFocus from "./VisualizationFocus.svelte"
// etc.

<KalanChart0/>
<Compare_1 />
<VisualizationDumbbell />